Dataset Download

In [2]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-09-22 10:00:54--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)...67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M   327KB/s    in 6m 0s   

2020-09-22 10:06:57 (435 KB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv
  inflating: ratings.csv
  inflating: README.txt              
  inflating: tags.csv


In [1]:
#Dataframe manipulation library
import pandas as pd
#Math functions, we'll only need the sqrt function so let's import only that
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Reading movies csv file

In [3]:
movies_df = pd.read_csv('movies.csv')

extract year from title and create year in a separate column

In [4]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df.head(1)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)


Extract year without parenthesis from year column

In [5]:
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df.head(1)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995


removing year from title

In [6]:
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))','')
movies_df.head(1)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995


let's also split the values in the Genres column into a list of Genres to simplify future use. This can be achieved by applying Python's split string function on the correct column.

In [7]:
movies_df['genres'] = movies_df.genres.str.replace('|',' ')
movies_df.head(1)

,movieId,title,genres,year
0,1,Toy Story,Adventure Animation Children Comedy Fantasy,1995


In [8]:
movies_df['title'] = movies_df['title'].str.lower()
movies_df['genres'] = movies_df['genres'].str.lower()
movies_df.head(2)

,movieId,title,genres,year
0,1,toy story,adventure animation children comedy fantasy,1995
1,2,jumanji,adventure children fantasy,1995


In [9]:
movies_df['title'][1]

'jumanji '

In [10]:
movies_df['title'] = movies_df['title'].apply(lambda x : x[:-1])

In [11]:
movies_df['title'][1]

'jumanji'

In [12]:
movies_df.isna().sum()

movieId     0
title       0
genres      0
year       68
dtype: int64

In [13]:
movies_df = movies_df.dropna(how='any')

In [14]:
movies_df.isna().sum()

movieId    0
title      0
genres     0
year       0
dtype: int64

In [15]:
movies_df['year'] = movies_df["year"].astype(int)

In [16]:
movies_df['year'].value_counts().sort_index().head(1)

1874    1
Name: year, dtype: int64

In [17]:
mask = (movies_df['year'] > 2010)
type(movies_df['year'][0])

numpy.int64

In [18]:
movies_df = movies_df.loc[mask].reset_index()

In [19]:
movies_df.tail(2)

,index,movieId,title,genres,year
6661,34202,151695,the survivalist,drama sci-fi thriller,2015
6662,34206,151709,zero,drama sci-fi,2015


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
cv = CountVectorizer()
count_matrix = cv.fit_transform(movies_df['genres'])
similarity = cosine_similarity(count_matrix)

In [24]:
def rcmd(m):
    m = m.lower()
    movies_df.head()
    similarity.shape
    
    if m not in movies_df['title'].unique():
        return('Sorry! The movie you requested is not in our database. Please check the spelling or try with some other movies')
    else:
        i = movies_df.loc[movies_df['title']==m].index[0]
        print(i)
        lst = list(enumerate(similarity[i]))
        
        lst = sorted(lst, key = lambda x:x[1] ,reverse=True)
        lst = lst[1:11] # excluding first item since it is the requested movie itself
        l = []
        for i in range(len(lst)):
            a = lst[i][0]
            l.append(movies_df['title'][a])
        return l

In [25]:
rcmd("Zero")

3968


['mildred pierce',
 'beaver, the',
 'kid with a bike, the (le gamin au vélo)',
 'tree of life, the',
 'fly away',
 'cinema verite',
 'too big to fail',
 'snow flower and the secret fan',
 'cyberbully',
 'help, the']

In [27]:
movies_df.to_csv('main_data.csv',index=False)
